# Análise comparativa

In [1]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1. Obtenção de dados

In [2]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,mpg,Medida utilizada para determinar a eficiência ...,quantitativa,contínua
1,cylinders,Quantidade de cilindros,quantitativa,discreta
2,displacement,Se refere ao volume de gás sugado ou descarreg...,quantitativa,contínua
3,horsepower,Representa a quantidade de trabalho que um mot...,quantitativa,contínua
4,weight,Peso do veículo,quantitativa,contínua
5,acceleration,Mecanismo que permite ao motorista aumentar ou...,quantitativa,contínua
6,model_year,Ano de fabricação,quantitativa,discreta
7,origin,País de origem,qualitativa,nominal
8,name,Nome do modelo,qualitativa,nominal


## 2. Preparação de dados

In [3]:
target_column = 'mpg'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_column")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'contínua'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

#### Tratamento de dados discrepantes

In [4]:
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),
    ('normalization', StandardScaler())
])

discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),
    ('normalization', StandardScaler())
])

continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),
    ('normalization', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

model = LogisticRegression()

## 3. Seleção de modelos
Iremos análisar quatro modelos, que serão testados utilizando um método de validação, a saber:

* K-Nearest-Neighbors
* Support Vector Machine
* Decision Tree
* Random Forest

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

* Acurácia (accuracy): proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
* Precisão (precision): proporção entre dados corretamente previstos como positivos e o total de observações positivas.
* Recall: proporção entre dados corretamente previstos como positivos com o total de observações.
* F1-score: média entre precision e recall, portanto levando em conta tanto falsos positivos quanto falsos negativos.

In [5]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

max_iter = 1000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),
    ('Suport Vector Machines', SVC(random_state=random_state, max_iter=max_iter), {"kernel": ["linear", "rbf"], 'C':[1,10,100,1000],'gamma':[0.0001, 0.001, 0.1, 1]}),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
]